In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import time
import pathlib

from datetime import datetime, time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from ipywidgets import IntProgress
from IPython.display import display
import time


In [3]:
max_count_progress = 100

In [4]:
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('vader_lexicon')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vaibhav\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\vaibhav\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\vaibhav\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vaibhav\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
BASE_DIR = pathlib.Path().cwd()

In [10]:
df_calls = pd.read_csv('calls.csv')
df_customers = pd.read_csv('customers.csv')

join_column = 'customer_id'

df1 = pd.merge(df_calls, df_customers, on=join_column, how='outer')

In [11]:
df_reason = pd.read_csv('reason.csv')
df_sentiment = pd.read_csv('sentiment_statistics.csv')

join_column = 'call_id'

df2 = pd.merge(df_reason, df_sentiment, on=join_column, how='outer')

In [12]:
join_column = 'call_id'

df = pd.merge(df1, df2, on=join_column, how='outer')

In [13]:
df = df.drop(columns=['agent_id_y'])
df = df.rename(columns={'agent_id_x': 'agent_id'})

In [14]:
# AHT calculation
df['call_start_datetime'] = pd.to_datetime(df['call_start_datetime'])
df['call_end_datetime'] = pd.to_datetime(df['call_end_datetime'])
df['handle_time'] = (df['call_end_datetime'] - df['call_start_datetime']).dt.total_seconds()

In [15]:
# AST calculation
df['agent_assigned_datetime'] = pd.to_datetime(df['agent_assigned_datetime'])
df['speed_to_answer'] = (df['agent_assigned_datetime'] - df['call_start_datetime']).dt.total_seconds()

In [16]:
df

,call_id,customer_id,agent_id,call_start_datetime,agent_assigned_datetime,call_end_datetime,call_transcript,customer_name,elite_level_code,primary_call_reason,agent_tone,customer_tone,average_sentiment,silence_percent_average,handle_time,speed_to_answer
0,131642,103262588,132311,2024-08-04 17:46:00,2024-08-04 17:55:00,2024-08-04 17:59:00,\n\nAgent: Thank you for calling United Airlin...,Michael Petersen,0.0,Seating,neutral,polite,-0.07,0.03,780.0,540.0
1,325527,8994245383,607742,2024-08-18 07:41:00,2024-08-18 07:45:00,2024-08-18 07:50:00,\n\nAgent: Thank you for calling United Airlin...,Andrew Hess,NaN,IRROPS,neutral,angry,-0.19,0.05,540.0,240.0
2,526871,5769982111,748700,2024-08-10 11:59:00,2024-08-10 12:08:00,2024-08-10 12:15:00,\n\nAgent: Thank you for calling United Airlin...,Joseph Thomas,3.0,Voluntary Change,neutral,frustrated,-0.22,0.13,960.0,540.0
3,634548,9534089270,542034,2024-08-11 11:15:00,2024-08-11 11:25:00,2024-08-11 11:39:00,\n\nAgent: Thank you for calling United Airlin...,Victoria Green,0.0,Seating,neutral,polite,-0.11,0.24,1440.0,600.0
4,693921,3046806067,633922,2024-08-06 15:58:00,2024-08-06 16:09:00,2024-08-06 16:15:00,\n\nAgent: Thank you for calling United Airlin...,Vickie Lopez,1.0,Voluntary Change,neutral,frustrated,-0.02,0.14,1020.0,660.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71805,9999337789,915905859,418648,2024-08-21 23:08:00,2024-08-21 23:17:00,2024-08-21 23:24:00,\n\nAgent: Thank you for calling United Airlin...,Jose Black,NaN,NaN,neutral,neutral,-0.05,0.15,960.0,540.0
71806,9999524523,3888680689,901589,2024-08-20 17:30:00,2024-08-20 17:36:00,2024-08-20 17:39:00,\n\nAgent: Thank you for calling United Airlin...,Angela Johnson,1.0,IRROPS,neutral,calm,-0.05,0.21,540.0,360.0
71807,9999623753,4101455140,835105,2024-08-03 14:07:00,2024-08-03 14:11:00,2024-08-03 14:28:00,\n\nAgent: Thank you for calling United Airlin...,Michael Reed,NaN,IRROPS,neutral,neutral,-0.17,0.40,1260.0,240.0
71808,9999669038,3173856584,819946,2024-08-19 12:50:00,2024-08-19 12:55:00,2024-08-19 13:35:00,\n\nAgent: Thank you for calling United Airlin...,Marvin White,3.0,Mileage Plus,calm,angry,0.07,0.52,2700.0,300.0


In [17]:
unique_sentiments = df['agent_tone'].unique()

unique_sentiments

array(['neutral', 'frustrated', 'calm', 'angry', nan, 'polite'],
      dtype=object)

In [18]:
#from sklearn.preprocessing import LabelEncoder
#le = LabelEncoder()

# we sort the sentiments from calmest to angriest manually, no machine learning here, and label those indices
tone_convert = dict(zip(unique_sentiments, [0, 2, -1, 1, 0.001, -2]))
tone_convert


{'neutral': 0,
 'frustrated': 2,
 'calm': -1,
 'angry': 1,
 nan: 0.001,
 'polite': -2}

In [19]:
df['agent_tone'] = df['agent_tone'].replace(tone_convert)
df['customer_tone'] = df['customer_tone'].replace(tone_convert)

df['elite_level_code'] = df['elite_level_code'].fillna(-1)

C:\Users\vaibhav\AppData\Local\Temp\ipykernel_4120\3894357191.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['agent_tone'] = df['agent_tone'].replace(tone_convert)
C:\Users\vaibhav\AppData\Local\Temp\ipykernel_4120\3894357191.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['customer_tone'] = df['customer_tone'].replace(tone_convert)


In [20]:
df.to_csv('base_table.csv')
df

,call_id,customer_id,agent_id,call_start_datetime,agent_assigned_datetime,call_end_datetime,call_transcript,customer_name,elite_level_code,primary_call_reason,agent_tone,customer_tone,average_sentiment,silence_percent_average,handle_time,speed_to_answer
0,131642,103262588,132311,2024-08-04 17:46:00,2024-08-04 17:55:00,2024-08-04 17:59:00,\n\nAgent: Thank you for calling United Airlin...,Michael Petersen,0.0,Seating,0.0,-2,-0.07,0.03,780.0,540.0
1,325527,8994245383,607742,2024-08-18 07:41:00,2024-08-18 07:45:00,2024-08-18 07:50:00,\n\nAgent: Thank you for calling United Airlin...,Andrew Hess,-1.0,IRROPS,0.0,1,-0.19,0.05,540.0,240.0
2,526871,5769982111,748700,2024-08-10 11:59:00,2024-08-10 12:08:00,2024-08-10 12:15:00,\n\nAgent: Thank you for calling United Airlin...,Joseph Thomas,3.0,Voluntary Change,0.0,2,-0.22,0.13,960.0,540.0
3,634548,9534089270,542034,2024-08-11 11:15:00,2024-08-11 11:25:00,2024-08-11 11:39:00,\n\nAgent: Thank you for calling United Airlin...,Victoria Green,0.0,Seating,0.0,-2,-0.11,0.24,1440.0,600.0
4,693921,3046806067,633922,2024-08-06 15:58:00,2024-08-06 16:09:00,2024-08-06 16:15:00,\n\nAgent: Thank you for calling United Airlin...,Vickie Lopez,1.0,Voluntary Change,0.0,2,-0.02,0.14,1020.0,660.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71805,9999337789,915905859,418648,2024-08-21 23:08:00,2024-08-21 23:17:00,2024-08-21 23:24:00,\n\nAgent: Thank you for calling United Airlin...,Jose Black,-1.0,NaN,0.0,0,-0.05,0.15,960.0,540.0
71806,9999524523,3888680689,901589,2024-08-20 17:30:00,2024-08-20 17:36:00,2024-08-20 17:39:00,\n\nAgent: Thank you for calling United Airlin...,Angela Johnson,1.0,IRROPS,0.0,-1,-0.05,0.21,540.0,360.0
71807,9999623753,4101455140,835105,2024-08-03 14:07:00,2024-08-03 14:11:00,2024-08-03 14:28:00,\n\nAgent: Thank you for calling United Airlin...,Michael Reed,-1.0,IRROPS,0.0,0,-0.17,0.40,1260.0,240.0
71808,9999669038,3173856584,819946,2024-08-19 12:50:00,2024-08-19 12:55:00,2024-08-19 13:35:00,\n\nAgent: Thank you for calling United Airlin...,Marvin White,3.0,Mileage Plus,-1.0,1,0.07,0.52,2700.0,300.0
